In [144]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [145]:
df = pd.read_csv("spam.csv", encoding="latin-1")[['v1','v2']]
df.columns = ['label','message']
df['is_spam'] = df['label'].map({'ham':0,'spam':1})
df['word_freq_free'] = df['message'].str.lower().str.count('free')
df['word_freq_win'] = df['message'].str.lower().str.count('win')
df['word_freq_offer'] = df['message'].str.lower().str.count('offer')
df['sms_length'] = df['message'].str.len()
df_features = df[['word_freq_free','word_freq_win','word_freq_offer','sms_length','is_spam']]

In [146]:
df_features.head()

,word_freq_free,word_freq_win,word_freq_offer,sms_length,is_spam
0,0,0,0,111,0
1,0,0,0,29,0
2,1,1,0,155,1
3,0,0,0,49,0
4,0,0,0,61,0


In [147]:
df_features.isna().sum()

word_freq_free     0
word_freq_win      0
word_freq_offer    0
sms_length         0
is_spam            0
dtype: int64

In [148]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   word_freq_free   5572 non-null   int64
 1   word_freq_win    5572 non-null   int64
 2   word_freq_offer  5572 non-null   int64
 3   sms_length       5572 non-null   int64
 4   is_spam          5572 non-null   int64
dtypes: int64(5)
memory usage: 217.8 KB


In [149]:
df_features.describe()

,word_freq_free,word_freq_win,word_freq_offer,sms_length,is_spam
count,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000
mean,0.058686,0.033740,0.009332,80.118808,0.134063
std,0.286039,0.203036,0.106775,59.690841,0.340751
min,0.000000,0.000000,0.000000,2.000000,0.000000
25%,0.000000,0.000000,0.000000,36.000000,0.000000
50%,0.000000,0.000000,0.000000,61.000000,0.000000
75%,0.000000,0.000000,0.000000,121.000000,0.000000
max,3.000000,3.000000,2.000000,910.000000,1.000000


In [150]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import GridSearchCV

In [151]:
X=df_features[['word_freq_free','word_freq_win','word_freq_offer','sms_length']]
y=df_features['is_spam']

In [152]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [153]:
model=MultinomialNB()
model.fit(x_train,y_train)
pred_y=model.predict(x_test)
accuracy_score(y_test,pred_y)

0.8923766816143498

In [154]:
model2=GaussianNB()
model2.fit(x_train,y_train)
pred_y2=model2.predict(x_test)
accuracy_score(y_test,pred_y2)

0.8896860986547085

In [155]:
model3=BernoulliNB()
model3.fit(x_train,y_train)
pred_y2=model3.predict(x_test)
accuracy_score(y_test,pred_y2)

0.8887892376681614

In [156]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [157]:
param_grid = {
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'criterion': ['gini', 'entropy'],
     'min_samples_split': [2,3,4, 5,8,9, 10,13,17,18, 20],
}


In [158]:
best=GridSearchCV(param_grid=param_grid,estimator=DecisionTreeClassifier(),cv=5)
best.fit(x_train,y_train)
best.best_params_   

{'criterion': 'entropy',
 'max_depth': 7,
 'min_samples_leaf': 2,
 'min_samples_split': 13}

In [ ]:
model_dt=DecisionTreeClassifier(max_depth=7,min_samples_leaf=2,criterion='entropy',min_samples_split=13)
model_dt.fit(x_train,y_train)
pred_y_dt=model_dt.predict(x_test)
accuracy_score(y_test,pred_y_dt)

0.9094170403587444

In [160]:
model_rf=RandomForestClassifier(n_estimators=100,max_depth=100)
model_rf.fit(x_train,y_train)
pred_y_rf=model_rf.predict(x_test)
accuracy_score(y_test,pred_y_rf)

0.8959641255605382

In [161]:
import pickle
pickle.dump(model_rf,open('spam_model.pkl','wb'))